# LangGraph Memory

Memory is a system that remembers information about previous interactions. For AI agents memory is crucial because it lets them remember previous interactions, learn from feedback, and adapt to user preferences.

# Memory Based on Scope
1. **Short-Term Memory (Thread-Scope):** It tracks the ongoing interactions by maintaining message history. LangGraph manages short-term memory as a part of the agent's state. state is written to a database using `checkpointer` so the thread can be resumed at any time. short-term memory updates when the graph is invoked or a step(node) is completed.
2. **Long-Term Memory (Across Threads):** It stores user-specific or application-level data across different sessions and is shared across conversational threads. Memories are scoped to any custom `namespace`, not just within a single `thread_id`. LangGraph provides `store` to let us save and recall long-term memories.

---
### 1. Short-Term Memory (Checkpoints/Persistence):
Short-term memory operates within individual conversation thread. It saves a snapshot of the graph at each step.

Key components are:
- **Threads:** A thread is a unique identifier that groups related checkpoints(snapshots) together.
- **StateSnapshot:** The actual checkpoint object that contains the graph state at a specific point in time.

Use cases are:
- **Conversation History:** Maintain chat context within a session.
- **Human-in-the-Loop:** Allowing human to inspect and modify state.
- **Time Travel:** Replaying or forking execution from specific points.
- **Fault Tolerance:** Resuming from the last successful step after failures
- **State Management:** Preserving intermediate results and artifacts

### Example

In [1]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver

class State(TypedDict):
    question: str
    answer: str

def node_1(state: State) -> State:
    return {"answer": "bye"}

workflow = StateGraph(State)
workflow.add_node("node_1", node_1)
workflow.add_edge(START, "node_1")
workflow.add_edge("node_1", END)

checkpointer = InMemorySaver()
graph = workflow.compile(checkpointer=checkpointer)

# Each invocation must specify a thread_id
config = {"configurable": {"thread_id": "conversation_1"}}
graph.invoke({"question": "Hello"}, config)

{'question': 'Hello', 'answer': 'bye'}

In [2]:
graph.get_state(config)

StateSnapshot(values={'question': 'Hello', 'answer': 'bye'}, next=(), config={'configurable': {'thread_id': 'conversation_1', 'checkpoint_ns': '', 'checkpoint_id': '1f09d515-1d42-6326-8001-8668505be8ac'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-09-29T16:28:23.925426+00:00', parent_config={'configurable': {'thread_id': 'conversation_1', 'checkpoint_ns': '', 'checkpoint_id': '1f09d515-1d41-66f6-8000-38fd5fe0e21d'}}, tasks=(), interrupts=())

---
### 2. Long-Term Memory (store):
Long-Term memory allows system to retain information across different conversations or sessions. instead of `thread`, long-term memory is saved with in a `namespace`.

### Long-Term Memory Types
1. **Semantic (Facts):** Facts about the user.
2. **Episodic (Experiences):** Past agent actions.
3. **Procedural (Rules):** Agent's system prompt.

### Semantic Memory:
This memory is often used to store user-specific information to personalize the agent's behavior by remembering facts and concepts from the past interactions. Semantic memory is managed in different ways; it can be a single, continuously updated key-value pair(`profile`) just like JSON.

### Episodic Memory:
Facts can be written to semantic memory, whereas experiences can be written to episodic memory. It is often used to help the agent remember how to accomplish a task.

### Procedural Memory:
It is used to remember the rules used to perform a task. For AI agents, procedural memory is a combination of model weights, agent code, and agent's prompt that collectively determine the agent's functionality.

In practice, it is fairly uncommon for agents to modify their model weights or rewrite their code. However, it is more common for agents to modify their own prompts.



### Example

LangGraph stores memories as JSON document organized using two key concepts:
1. Namespace: A tuple that acts as a folder structure for organizing related memories. `("peyman_kh", "profile")`
2. Key: A unique identifier for a memory, just like a filename.

In [3]:
# Import library
from langgraph.store.memory import InMemoryStore

# Namespace (similar to directory)
user_id = "user_123"
namespace = (user_id, "profile")  #/user_123/profile

# Key (similar to filename)
key = "user_profile"

# Value (similar to file content)
value = {
    "name": "Peyman Kh",
    "age": 30,
    "interests": ["travel", "cooking", "reading"]
}

# Save to memory
store = InMemoryStore()
store.put(namespace, key, value)

In [4]:
# Read from memory
store.get(namespace, key)

Item(namespace=['user_123', 'profile'], key='user_profile', value={'name': 'Peyman Kh', 'age': 30, 'interests': ['travel', 'cooking', 'reading']}, created_at='2025-09-29T16:28:23.947994+00:00', updated_at='2025-09-29T16:28:23.947997+00:00')

---

## Trustcall

When we want to update memory, two major problems might happen:
1. When the memory schema is complex, LLM might give a validation error.
2. If we rewrite memory everytime, we might miss context.

That is the motivation behind Trustcall open-source library.

In [5]:
# Import libraries
from typing import Optional
from pydantic import BaseModel, Field
from trustcall import create_extractor
from langchain_openai import ChatOpenAI
from codes.config.config import config

# Create memory schema
class UserProfile(BaseModel):
    name: Optional[str] = Field(
        None,
        description="The name of the user."
    )
    age: Optional[int] = Field(
        None,
        description="The age of the user."
    )
    gender: Optional[str] = Field(
        None,
    ),
    interests: Optional[list[str]] = Field(
        None,
        description="The user's interests."
    )

# Create ChatModel instance
llm = ChatOpenAI(api_key=config.openai_api_key.get_secret_value(), model="gpt-4o")

# Create trustcall executor instead of llm.with_structured_output()
extractor = create_extractor(
    llm,
    tools=[UserProfile],
    tool_choice="UserProfile"
)

result = extractor.invoke({
    "messages": [{"role": "user", "content": "I'm Peyman, 27, love coding and gaming"}]
})

2025-09-29 19:28:24,257 - root - INFO - Configuration loaded for environment: development
/Users/peyman/Documents/github/llm-engineering-12-week-roadmap/.venv/lib/python3.12/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=False, default=None),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
2025-09-29 19:28:24,499 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'headers': {'X-Stainless-Raw-Response': 'true'}, 'files': None, 'idempotency_key': 'stainless-python-retry-dceb1e87-a09e-440e-b42a-30455c946756', 'json_data': {'messages': [{'content': "I'm Peyman, 27, love coding and gaming", 'role': 'user'}], 'model': 'gpt-4o', 'stream': False, 'tool_choice': {'type': 'function', 'function': {'name': 'UserProfile'}}, 'tools': [{'type': 'function', 'function': {'name': 'UserProfi

In [6]:
for msg in result["messages"]:
    msg.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  UserProfile (call_oMBBpxG56Yry0qF9FzkyQac6)
 Call ID: call_oMBBpxG56Yry0qF9FzkyQac6
  Args:
    name: Peyman
    age: 27
    interests: ['coding', 'gaming']


In [7]:
result["responses"]

[UserProfile(name='Peyman', age=27, gender=(FieldInfo(annotation=NoneType, required=False, default=None),), interests=['coding', 'gaming'])]

In [8]:
result["responses"][0].model_dump()

/Users/peyman/Documents/github/llm-engineering-12-week-roadmap/.venv/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [input_value=(FieldInfo(annotation=Non...d=False, default=None),), input_type=tuple])
  return self.__pydantic_serializer__.to_python(


{'name': 'Peyman',
 'age': 27,
 'gender': (FieldInfo(annotation=NoneType, required=False, default=None),),
 'interests': ['coding', 'gaming']}

## Partial Update with Trustcall

In [9]:
from langchain_core.messages import HumanMessage

message = HumanMessage(content="Some times I also go finishing with my family")
updated_memory = extractor.invoke({
    "messages": [message],
    "existing": {"UserProfile": result["responses"][0].model_dump()}
})

/Users/peyman/Documents/github/llm-engineering-12-week-roadmap/.venv/lib/python3.12/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=False, default=None),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
2025-09-29 19:28:39,168 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'headers': {'X-Stainless-Raw-Response': 'true'}, 'files': None, 'idempotency_key': 'stainless-python-retry-7b8f7219-fa86-4f70-9440-cd16b7180626', 'json_data': {'messages': [{'content': 'Generate JSONPatches to update the existing schema instances.\n<existing>\n<schema id=UserProfile>\n<instance>\n{\'name\': \'Peyman\', \'age\': 27, \'gender\': (FieldInfo(annotation=NoneType, required=False, default=None),), \'interests\': [\'coding\', \'gaming\']}\n</instance>\n    <json_schema>\n    {\'properties\': {

In [10]:
updated_memory["responses"]

[]

---

## Memory Collection Schema

Sometimes we want the memory to be a list of documents instead of a single document such as UserProfile.

In [11]:
from pydantic import BaseModel, Field

class Memory(BaseModel):
    content: str = Field(description="The main content of the memory. For example: User expressed interest in learning about French.")

class MemoryCollection(BaseModel):
    memories: list[Memory] = Field(description="A list of memories about the user.")

In [12]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

trustcall_executor = create_extractor(
    llm,
    tools=[Memory],
    tool_choice="Memory",
    enable_inserts=True,  # Allow the extractor to insert new memories to the collection
)

prompt = "Extract memories from the following conversation:"

conversation = [HumanMessage(content="Hi, I'm Peyman."),
                AIMessage(content="Nice to meet you, Peyman."),
                HumanMessage(content="This morning I have started my new project with langgraph.")]

result = trustcall_executor.invoke({"messages": [SystemMessage(content=prompt)] + conversation})

2025-09-29 19:28:42,415 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'headers': {'X-Stainless-Raw-Response': 'true'}, 'files': None, 'idempotency_key': 'stainless-python-retry-3208a9ab-0afb-405f-93d4-777ebdb34e5a', 'json_data': {'messages': [{'content': 'Extract memories from the following conversation:', 'role': 'system'}, {'content': "Hi, I'm Peyman.", 'role': 'user'}, {'content': 'Nice to meet you, Peyman.', 'role': 'assistant'}, {'content': 'This morning I have started my new project with langgraph.', 'role': 'user'}], 'model': 'gpt-4o', 'stream': False, 'tool_choice': {'type': 'function', 'function': {'name': 'Memory'}}, 'tools': [{'type': 'function', 'function': {'name': 'Memory', 'description': None, 'parameters': {'properties': {'content': {'description': 'The main content of the memory. For example: User expressed interest in learning about French.', 'title': 'Content', 'type': 'string'}}, 'required': ['content'], 'title': 'Me

In [13]:
for msg in result["messages"]:
    msg.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  Memory (call_5CPNwA5i0OvFmVtiXdX7x4Ln)
 Call ID: call_5CPNwA5i0OvFmVtiXdX7x4Ln
  Args:
    content: Peyman started a new project with langgraph this morning.


In [14]:
for msg in result["responses"]:
    print(msg)

content='Peyman started a new project with langgraph this morning.'


In [15]:
for msg in result["response_metadata"]:
    print(msg)

{'id': 'call_5CPNwA5i0OvFmVtiXdX7x4Ln'}


In [16]:
updated_conversation = [AIMessage(content="That's great, tell me more about it?"),
                        HumanMessage(content="Actually it is a mobile app developed with Flutter, and backend is python."),
                        AIMessage(content="Nice, What else is on your mind?"),
                        HumanMessage(content="I was thinking so I need to start designing the database schema and design entities and relationships."),]

# Update the instruction
system_msg = """Update existing memories and create new ones based on the following conversation:"""

tool_name = "Memory"
existing_memory = []
for index, content in enumerate(result["responses"]):
    existing_memory.append((str(index), "Memory", content))

In [17]:
existing_memory

[('0',
  'Memory',
  Memory(content='Peyman started a new project with langgraph this morning.'))]

In [18]:
result_new = trustcall_executor.invoke({
    "messages": [SystemMessage(content=system_msg)] + updated_conversation,
    "existing": existing_memory
})

2025-09-29 19:28:43,269 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'headers': {'X-Stainless-Raw-Response': 'true'}, 'files': None, 'idempotency_key': 'stainless-python-retry-2748671b-2161-4396-a5f6-5605e0f36bf3', 'json_data': {'messages': [{'content': 'Update existing memories and create new ones based on the following conversation:\n\nGenerate JSONPatches to update the existing schema instances. If you need to extract or insert *new* instances of the schemas, call the relevant function(s).\n<existing>\n<instance id=0 schema_type="Memory">\n{\'content\': \'Peyman started a new project with langgraph this morning.\'}\n</instance>\n</existing>\n', 'role': 'system'}, {'content': "That's great, tell me more about it?", 'role': 'assistant'}, {'content': 'Actually it is a mobile app developed with Flutter, and backend is python.', 'role': 'user'}, {'content': 'Nice, What else is on your mind?', 'role': 'assistant'}, {'content': 'I was thin

In [19]:
for m in result_new["responses"]:
    print(m)

content='Peyman started a new project with langgraph this morning. The project is a mobile app developed with Flutter, and the backend is Python.'
content='Peyman is thinking about designing the database schema and designing entities and relationships for the new project.'


In [20]:
for msg in result_new["messages"]:
    msg.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  Memory (call_rfQb3AMle8ZYBDcsEmP30loj)
 Call ID: call_rfQb3AMle8ZYBDcsEmP30loj
  Args:
    content: Peyman started a new project with langgraph this morning. The project is a mobile app developed with Flutter, and the backend is Python.
  Memory (call_afO64ITnGQXAMgCNKinhcEhj)
 Call ID: call_afO64ITnGQXAMgCNKinhcEhj
  Args:
    content: Peyman is thinking about designing the database schema and designing entities and relationships for the new project.


In [21]:
for msg in result_new["response_metadata"]:
    print(msg)

{'id': 'call_rfQb3AMle8ZYBDcsEmP30loj', 'json_doc_id': '0'}
{'id': 'call_afO64ITnGQXAMgCNKinhcEhj'}


In [25]:
existing_memory = [(str(index), "Memory", memory.model_dump()) for index, memory in enumerate(result_new["responses"])]

existing_memory

[('0',
  'Memory',
  {'content': 'Peyman started a new project with langgraph this morning. The project is a mobile app developed with Flutter, and the backend is Python.'}),
 ('1',
  'Memory',
  {'content': 'Peyman is thinking about designing the database schema and designing entities and relationships for the new project.'})]